In [ ]:
from ipywidgets import *

In [ ]:
# Objet exemple
class ObjectWithClasses:
    def __init__(self, values):
        self.value = dict(values)

    #def add_cell_class(self, cl) # Dans l'adapter ?
    #    if cl not in self.cell_classes:
    #        self.cell_classes.append(cl)
            #self.apply_classes()

    # fonction bas niveau pour rajouter du style aux cellules vérifiant la propriété `prop`
    #def set_class_attr(self, cl, attr, value):
    #    self.add_property(prop)
    #    self.style_placeholder.set_class(self.cell_widget_class.css_classes + (prop.get_css_name(),), attr, value)



In [ ]:
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
from sage_combinat_widgets.grid_view_editor import GridViewEditor
from sage_combinat_widgets import *
ClOdd = CellClass('odd')
ClMin = CellClass('min')
ClInc = CellClass('diag_inc')
ClType = CellClass('type') #, cond=lambda x: x == i) for i in range(4) ]

In [ ]:
# Adapter pour l'objet exemple
class ObjectGridViewAdapter(GridViewAdapter):
    objclass = ObjectWithClasses
    celltype = int
    cellzero = False
    
    @staticmethod
    def compute_cells(obj):
        return obj.value
    
    # On définit ici les classes des cellules de l'objet
    @staticmethod
    def compute_cell_classes(obj):
        values = obj.value

        classes = {}

        for i,j in values:
            #print()
            #print(i,j)
            classes[i,j] = []
            # cl1: valeur impaire
            if values[i,j] & 1:
                #print("odd ok")
                classes[i,j].append((ClOdd, True))
            #print(classes[i,j])
            
            # cl2: minimum local
            for x,y in ((0,1),(0,-1), (1,0), (-1,0)):
                if (i+x,j+y) in values and values[i+x,j+y] < values[i,j]:
                    break
            else:
                #print("min ok")
                classes[i,j].append((ClMin, True))

            #print(classes[i,j])
            
            # cl3: plus petit que la valeur en bas à droite
            if (i+1,j+1) in values and values[i+1,j+1] > values[i,j]:
                #print("inc ok")
                classes[i,j].append((ClInc, True))

            #print(classes[i,j])

            # cl4: valeur entre 0 et 3, de paramètre la valeur en question
            if 0 <= values[i,j] < 4:
                #print("type ok")
                classes[i,j].append((ClType, values[i,j]))
            #print(classes[i,j])

        return classes
    
    @classmethod
    def from_cells(self, cells):
        return ObjectWithClasses(cells)

    @staticmethod
    def addable_cells(obj):
        if obj.value:
            w = max(i for i,j in obj.value)+1
            h = max(j for i,j in obj.value)+1
            return list(set((i,j) for i in range(w+1) for j in range(h+1)) - set(obj.value))
        return [(0,0)]
    
    @staticmethod
    def removable_cells(obj):
        return list(obj.value)
    
    def add_cell(self, obj, pos, value, dirty={}):
        obj = ObjectWithClasses(obj.value)
        obj.value[pos] = value
        return obj
    
    def remove_cell(self, obj, pos, dirty={}):
        obj = ObjectWithClasses(obj.value)
        del obj.value[pos]
        return obj

In [ ]:
%%html
<style>
.odd_7 { background-color: green }
.min_7 { color: red }
</style>
<div class="odd_7"><input type="text"></div>
<div class="min_7"><input type="text"></div>

In [ ]:
from ipywidgets import *
from sage_combinat_widgets.grid_view_widget import apply_css
t1 = Text(_dom_classes=["min_7"])
apply_css(".min_7 { background-color: pink }")
t1

In [ ]:
t2 = t.add_class("min_7")
#t.add_class??
t2._dom_classes
t2

In [ ]:
cell_layout = Layout(width='28px')
obj = ObjectWithClasses({(0,0):1})
g = GridViewWidget(obj, ObjectGridViewAdapter())
g.update_style()
set_cell_class_attr(ClOdd, 'background-color', 'green')
set_cell_class_attr(ClMin, 'color', 'red')
g

In [ ]:
g.value = ObjectWithClasses( {(0,0):2,(0,1):5} )

In [ ]:
#g.__dict__

In [ ]:
%%html
<style>
.odd_7FF6D1A48750 { background-color:green }
.min_7FF6D1A48750 { color:red }
</style>

In [ ]:
#for cl in ClTypes + [ClOdd, ClMin, ClInc]:
#    g.add_class_to_cell(cl)
#classes = g.get_cell_classes() # interrogera l'adapter
g.update_style()
set_cell_class_attr(ClOdd, 'background-color', 'green')
set_cell_class_attr(ClMin, 'color', 'red')

In [ ]:
from sage.all import ParallelogramPolyomino, DyckWords
from sage_widget_adapters.combinat.parallelogram_polyomino_grid_view_adapter import ParallelogramPolyominoGridViewAdapter

In [ ]:
class ParallelogramPolyominoAdapter(ParallelogramPolyominoGridViewAdapter):
    @staticmethod
    def get_bounce_cells_from_pp(pp):
        ret = []
        x,y = 0,-1
        for i,v in enumerate(pp.bounce_path()):
            dx,dy = (1,0) if i & 1 else (0,1) 
            for j in range(v):
                x,y = x+dx, y+dy
                ret.append((i,(x,y)))
        return ret
    
    @staticmethod
    def get_properties(obj):
        props = {}
        
        # Values of the bounce path
        for i,pos in ParallelogramPolyominoAdapter.get_bounce_cells_from_pp(obj):
            props[pos] = [('bounce0', True)]

        return props

In [ ]:
pp = ParallelogramPolyomino.from_dyck_word(DyckWords(10).random_element())

In [ ]:
gp = GridViewWidget(pp, ParallelogramPolyominoAdapter(), cell_layout=cell_layout, 
                    cell_widget_class=DisabledButton, 
                    addable_widget_class=AddableButton)

In [ ]:
gp

In [ ]:
PropBounce = Property('bounce0')
gp.add_property(PropBounce)
gp.set_property_attr(PropBounce, 'background-color', 'red')

In [ ]:
gp.set_cell_layout(Layout(width='22px', height='22px'))